In [92]:
import random
import numpy as np
from IPython.display import clear_output
from keras.models import Sequential
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,BatchNormalization
import chess

def flatten(itr):
    t = tuple()
    for e in itr:
        try:
            t += flatten(e)
        except:
            t += (e,)
    return t

def sigmoid(x):
    return 1/(1+np.exp(-x))

class genetic_algorithm:
        
    def execute(pop_size,generations,threshold):
        class Agent:
            def __init__(self):
                class neural_network:
                    def __init__(self):
                        model = Sequential()
                        model.add(Dense(64, activation='relu', input_shape=(8,8,12)))
                        model.add(Flatten())
                        model.add(BatchNormalization())
                        model.add(Dense(1,activation = 'sigmoid'))
                        self.model = model
                        self.weights = model.get_weights()
                    def propagate(self,X):
                        self.model.predict(X)
                        return self.model.predict(X)
                    
                self.neural_network = neural_network()
                self.fitness = 0
            def __str__(self):
                    return 'Loss: ' + str(self.fitness)
        
                
        def generate_agents(population):
            return [Agent() for _ in range(population)]
        
        def fitness(agents):
            chess_dict = {
                'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
                'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
                'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
                'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
                'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
                'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
                'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
                'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
                'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
                'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
                'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
                'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
                '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
            }

                
            def make_matrix(board): 
                pgn = board.epd()
                foo = []  
                pieces = pgn.split(" ", 1)[0]
                rows = pieces.split("/")
                for row in rows:
                    foo2 = []  
                    for thing in row:
                        if thing.isdigit():
                            for i in range(0, int(thing)):
                                foo2.append('.')
                        else:
                            foo2.append(thing)
                    foo.append(foo2)
                return foo

            def translate(matrix,chess_dict):
                rows = []
                for row in matrix:
                    terms = []
                    for term in row:
                        terms.append(chess_dict[term])
                    rows.append(terms)
                return rows
            for i in range(len(agents)-1):
                player = [0]*2
                player[0] = agents[i]
                player[1] = agents[i+1]

                board = chess.Board()
                while not(board.is_game_over()):           
                    matrix = make_matrix(board)
                    X = np.reshape(np.array(translate(matrix,chess_dict)),(1,8,8,12))
                    if i % 2 == 0:
                        player_index = 0
                    else:
                        player_index = 1
                    move_ratio = player[player_index].neural_network.propagate(X)
                    legal_moves = str(board.legal_moves)[36:-2].replace(',','').split()
                    index = int(move_ratio * len(legal_moves))
                    board.push_san(legal_moves[index])

                def evaluate_score(board):
                    player_fitness = [0]*2
                    sides = [True,False]
                    result = board.result()
                    if result == '1/2-1/2':
                        player_fitness[0] += 10
                        player_fitness[1] += 10
                    elif result == '1-0':
                        player_fitness[0] += 100
                        player_fitness[1] -= 100
                    elif result == '0-1':
                        player_fitness[0] -= 100
                        player_fitness[1] += 100
                    def calculate_material(board,side):
                        material = 0
                        material_values = [1,3,3,5,6,0]
                        for i in range(1,7):
                            count = len(board.pieces(i,side))
                            material += count*material_values[i-1]
                        return material
                    for i in range(2):
                        player_fitness[i] += calculate_material(board,sides[i])
                    return player_fitness
    #                 for i in range(2):
    #                     sides[0]
                def calculate_material(board,side):
                    material = 0
                    material_values = [1,3,3,5,6,0]
                    for i in range(1,7):
                        board = chess.Board()
                        count = len(board.pieces(i,side))
                        material += count*material_values[i-1]
                    return material
                fitnesses = evaluate_score(board)
                player[0].fitness += fitnesses[0]
                player[1].fitness += fitnesses[1]
            return agents
        
        def selection(agents):
            agents = sorted(agents, key=lambda agent: agent.fitness, reverse=True)
            print('\n'.join(map(str, agents)))
            agents = agents[:int(0.2 * len(agents))]
            return agents
        
        def unflatten(flattened,shapes):
            newarray = []
            index = 0
            for shape in shapes:
                size = np.product(shape)
                newarray.append(flattened[index : index + size].reshape(shape))
                index += size
            return newarray
        
        def crossover(agents,pop_size):
            offspring = []
            for _ in range((pop_size - len(agents)) // 2):
                parent1 = random.choice(agents)
                parent2 = random.choice(agents)
                child1 = Agent()
                child2 = Agent()
                        
                
                shapes = [a.shape for a in parent1.neural_network.weights]
                
                genes1 = np.concatenate([a.flatten() for a in parent1.neural_network.weights])
                genes2 = np.concatenate([a.flatten() for a in parent2.neural_network.weights])
                
                split = random.randint(0,len(genes1)-1)

                child1_genes = np.array(genes1[0:split].tolist() + genes2[split:].tolist())
                child2_genes = np.array(genes1[0:split].tolist() + genes2[split:].tolist())
                
                child1.neural_network.weights = unflatten(child1_genes,shapes)
                child2.neural_network.weights = unflatten(child2_genes,shapes)
                
                offspring.append(child1)
                offspring.append(child2)
            agents.extend(offspring)
            return agents
        
        def mutation(agents):
            for agent in agents:
                if random.uniform(0.0, 1.0) <= 0.1:
                    weights = agent.neural_network.weights
                    shapes = [a.shape for a in weights]

                    flattened = np.concatenate([a.flatten() for a in weights])
                    randint = random.randint(0,len(flattened)-1)
                    flattened[randint] = np.random.randn()

                    newarray = []
                    indeweights = 0
                    for shape in shapes:
                        size = np.product(shape)
                        newarray.append(flattened[indeweights : indeweights + size].reshape(shape))
                        indeweights += size
                    agent.neural_network.weights = newarray
            return agents
        
        for i in range(generations):
            print('Generation',str(i),':')
            agents = generate_agents(pop_size)
            agents = fitness(agents)
            agents = selection(agents)
            agents = crossover(agents,pop_size)
            agents = mutation(agents)
            agents = fitness(agents)
            
            if any(agent.fitness > threshold for agent in agents):
                print('Threshold met at generation '+str(i)+' !')
                break
                
            clear_output()
        return agents[0]
ga = genetic_algorithm
agent = ga.execute(10,100,1000)
weights = agent.neural_network.weights
agent.fitness
# agent.neural_network.propagate(X)

137

In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
model = Sequential()
model.add(Dense(64,input_shape = (8,8,12)))
model.add(Flatten())
model.add(Dense(12))

In [6]:
model.summary()
weights = model.get_weights()
flatten(weights)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8, 8, 64)          832       
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                49164     
Total params: 49,996
Trainable params: 49,996
Non-trainable params: 0
_________________________________________________________________


(-0.21450955,
 0.1818482,
 -0.1324994,
 -0.25583774,
 0.0493598,
 0.05355555,
 0.20013991,
 -0.24384427,
 -0.012229383,
 0.2235882,
 -0.11038625,
 0.14760098,
 0.098683715,
 -0.17381021,
 -0.0992374,
 0.16306439,
 -0.17835775,
 0.026804566,
 -0.27202824,
 0.121613085,
 -0.11826512,
 0.03431046,
 0.13060585,
 0.06888035,
 -0.25016183,
 -0.27283928,
 0.17743525,
 -0.12338321,
 0.0755918,
 0.08405343,
 -0.18141201,
 -0.11137736,
 -0.05098127,
 -0.20983675,
 0.21638131,
 -0.24936758,
 0.11406052,
 -0.0061074495,
 0.059082568,
 -0.24442427,
 0.20246285,
 0.053631783,
 0.050997138,
 -0.073615596,
 -0.16772225,
 -0.001665622,
 0.15725228,
 -0.02888292,
 -0.1183784,
 -0.08571583,
 0.13167039,
 -0.2214,
 0.082980365,
 -0.038496777,
 -0.031651765,
 -0.20236558,
 0.27080336,
 -0.03734073,
 0.22809723,
 0.22005764,
 -0.17088443,
 0.1034995,
 -0.111451656,
 -0.2707971,
 0.19642001,
 0.12574843,
 -0.10187727,
 0.257984,
 0.06745836,
 0.16828254,
 -0.052120507,
 0.1842775,
 0.22747967,
 -0.053727314,

In [5]:
def flatten(itr):
    t = tuple()
    for e in itr:
        try:
            t += flatten(e)
        except:
            t += (e,)
    return t

In [15]:
shapes

[(3, 3, 12, 64), (64,), (3, 3, 64, 64), (64,), (256, 12), (12,)]

In [16]:
def _flatten(values):
    if isinstance(values, np.ndarray):
        yield values.flatten()
    else:
        for value in values:
            yield from _flatten(value)

def flatten(values):
    # flatten nested lists of np.ndarray to np.ndarray
    return np.concatenate(list(_flatten(values)))

def _unflatten(flat_values, prototype, offset):
    if isinstance(prototype, np.ndarray):
        shape = prototype.shape
        new_offset = offset + np.product(shape)
        value = flat_values[offset:new_offset].reshape(shape)
        return value, new_offset
    else:
        result = []
        for value in prototype:
            value, offset = _unflatten(flat_values, value, offset)
            result.append(value)
        return result, offset

def unflatten(flat_values, prototype):
    # unflatten np.ndarray to nested lists with structure of prototype
    result, offset = _unflatten(flat_values, prototype, 0)
    assert(offset == len(flat_values))
    return result

flatten(weights)

TypeError: 'numpy.float32' object is not iterable

In [38]:
board = chess.Board()
len(board.pieces(1,True))

8